In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import anndata as ad

In [2]:
sc._settings.ScanpyConfig.n_jobs = -1

In [2]:
def z_score_normalization(data):
    return (data - np.mean(data, axis=0)) / np.std(data, axis=0)

def reorder_columns(df, col_changed, col_position) -> pd.DataFrame:
    """
    Reordering colums. The second input can either be an integer for index or it can be a reference column name. If reference column name is chosen, the column will be placed after the reference column.
    """
    if not isinstance(df, pd.DataFrame):
        raise ValueError("The first argument must be a pandas DataFrame")
    if not isinstance(col_changed, str):
        raise ValueError("The second argument must be a string representing a column name")
    if not isinstance(col_position, (str, int)):
        raise ValueError("The third argument must be either a string representing a column name or an integer representing a column index")
    if isinstance(col_position, int):
        cols = df.columns.tolist()
        if col_changed in cols and col_position <= len(cols) + 1:
            cols.remove(col_changed)
        
            index = col_position
            cols.insert(index, col_changed)
            
            df = df[cols]
    else:
        cols = df.columns.tolist()
        if col_changed in cols and col_position in cols:
            cols.remove(col_changed)
            index = cols.index(col_position)
            cols.insert(index+1, col_changed)
            
            df = df[cols]
    return df

In [2]:
annot = pd.read_csv('/Volumes/Lukas_SSD/phenotyping_benchmark/datasets/lymphoma_CODEX/quantification/cells_annotation.csv')
exp = pd.read_csv('/Volumes/Lukas_SSD/phenotyping_benchmark/datasets/lymphoma_CODEX/quantification/cells_expression.csv')

In [ ]:
annot

In [ ]:
exp

In [3]:
df = exp.merge(annot, left_on='Unnamed: 0', right_on='unique_cell_id')

In [7]:
df.iloc[:100,:].to_csv('/Users/lukashat/Downloads/cells_expression_annot.csv', index=True)

In [ ]:
df.drop(columns=['Unnamed: 0','PatientID', 'unique_cell_id'], inplace=True)
df.rename(columns={'Merged_final': 'cell_type'}, inplace=True)
df.to_csv('/Volumes/Lukas_SSD/phenotyping_benchmark/datasets/lymphoma_CODEX/quantification/processed/lymphoma_24_cleaned.csv', index=False)

In [ ]:
df['unique_region'].unique()

In [ ]:
df

In [ ]:
X_columns = df.columns[:df.columns.get_loc('x')]
obs_columns = df.columns[df.columns.get_loc('x'):]
adata = ad.AnnData(
    X=df[X_columns],
    obs=df[obs_columns],
    var=pd.DataFrame(index=X_columns)
)

In [16]:
adata.raw = adata
adata.X = np.arcsinh(adata.X/150)
adata.layers['zscore'] = z_score_normalization(adata.X)

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10)
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color='cell_type')

In [ ]:
sc.pl.matrixplot(adata, var_names=adata.var_names, groupby='cell_type', cmap='vlag', dendrogram=True, standard_scale='var', layer='zscore')

In [ ]:
sc.tl.rank_genes_groups(adata, groupby='cell_type' , method='wilcoxon')

In [24]:
sc.tl.filter_rank_genes_groups(adata, min_in_group_fraction=0.2, max_out_group_fraction = 1, key = 'rank_genes_groups', key_added='rank_genes_groups_filtered')

In [ ]:
sc.pl.rank_genes_groups_dotplot(adata, n_genes=5, standard_scale='var', key='rank_genes_groups')